In [4]:
import sklearn
import pandas as pd

import logging

import torch
cuda_available = torch.cuda.is_available()

from simpletransformers.ner import NERModel, NERArgs

In [23]:
model_name="indobenchmark/indobert-lite-large-p1"

In [14]:
train_df=pd.read_csv("D:\Drive\KULEEAH\Semester 8\TA Farihin\Dataset\\bert\\train.csv")
eval_df=pd.read_csv("D:\Drive\KULEEAH\Semester 8\TA Farihin\Dataset\\bert\\val.csv")


In [15]:
train_df=train_df[:50010]
train_df

,sentence_id,words,labels
0,1382601382042103808,Hidup,O
1,1382601382042103808,sesedih,O
2,1382601382042103808,dan,O
3,1382601382042103808,secaper,O
4,1382601382042103808,apa,O
...,...,...,...
50005,1383190790147436548,gak,O
50006,1383190790147436548,ada,O
50007,1383190790147436548,kelas,O
50008,1383190790147436548,.,O


In [16]:
eval_df

,sentence_id,words,labels
0,1385573262067847169,Nadiem,B-PER
1,1385573262067847169,Makarim,I-PER
2,1385573262067847169,dg,O
3,1385573262067847169,Presiden,O
4,1385573262067847169,ke-5,O
...,...,...,...
27206,1382913410749190149,aku,O
27207,1383795163412058114,ekonomi,O
27208,1383795163412058114,",",O
27209,1383795163412058114,infrastruktur,O


In [17]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [18]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    # "num_train_epochs": 1,
    "num_train_epochs": 5,
    "weight_decay": 0,
    # "learning_rate": 4e-5,
    "learning_rate": 0.00006828,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,
}


In [19]:
# model = NERModel('bert', 
#                 'outputs',
#                 args=model_args,
#                 labels=label_list,
#                 use_cuda=cuda_available,
#                 )
# result, model_outputs, wrong_preds = model.eval_model(eval_df)
# result

In [26]:
model = NERModel('bert', 
                model_name,
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )

layer.1.attention.output.LayerNorm.bias', 'encoder.layer.13.intermediate.dense.bias', 'encoder.layer.19.output.dense.bias', 'encoder.layer.12.attention.self.key.bias', 'encoder.layer.19.attention.self.query.weight', 'encoder.layer.4.attention.output.LayerNorm.weight', 'encoder.layer.13.attention.self.query.weight', 'encoder.layer.20.attention.self.query.bias', 'encoder.layer.14.output.dense.weight', 'encoder.layer.21.intermediate.dense.bias', 'encoder.layer.8.output.LayerNorm.bias', 'encoder.layer.7.attention.output.LayerNorm.bias', 'encoder.layer.15.attention.self.query.bias', 'encoder.layer.22.attention.output.dense.weight', 'encoder.layer.16.attention.self.key.bias', 'encoder.layer.5.output.dense.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.21.attention.self.value.bias', 'encoder.layer.4.output.LayerNorm.bias', 'encoder.layer.3.attention.output.LayerNorm.bias', 'encoder.layer.8.attention.self.value.bias', 'encoder.layer.11.attention.output.LayerNorm.bias', 'e

RuntimeError: Error(s) in loading state_dict for BertForTokenClassification:
	size mismatch for embeddings.word_embeddings.weight: copying a param with shape torch.Size([30000, 128]) from checkpoint, the shape in current model is torch.Size([30000, 1024]).
	size mismatch for embeddings.position_embeddings.weight: copying a param with shape torch.Size([512, 128]) from checkpoint, the shape in current model is torch.Size([512, 1024]).
	size mismatch for embeddings.token_type_embeddings.weight: copying a param with shape torch.Size([2, 128]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
	size mismatch for embeddings.LayerNorm.weight: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for embeddings.LayerNorm.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([1024]).

In [ ]:
model.train_model(train_df,  eval_data=eval_df)

In [ ]:
result, model_outputs, wrong_preds = model.eval_model(eval_df)

In [12]:
result

{'eval_loss': 0.401481420783236,
 'precision': 0.46971153846153846,
 'recall': 0.40725302209253855,
 'f1_score': 0.4362580933244028}

In [15]:
# model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass, acc=accuracy_score)

Running Evaluation: 100%|██████████| 269/269 [00:48<00:00,  5.53it/s]
d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [16]:
predictions, raw_outputs = model.predict(
    ["@PolsekAlian PolriBerikan inovasi-inovasi Polri yang memudahkan masyarakat, kegiatan pemolisian masyarakat dan seluruh tindakan Kepolisian dalam menghadapi gangguan Kamtibmas, dan lain-lain - Jenderal Polisi Drs. Listyo Sigit Prabowo, https://t.co/f8GZ6VPqPD. - Kapolri #PolriTVRadioPresisi"]
        )

Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 20.84it/s]


In [17]:
predictions

[[{'@PolsekAlian': 'O'},
  {'PolriBerikan': 'O'},
  {'inovasi-inovasi': 'O'},
  {'Polri': 'B-ORG'},
  {'yang': 'O'},
  {'memudahkan': 'O'},
  {'masyarakat,': 'O'},
  {'kegiatan': 'O'},
  {'pemolisian': 'O'},
  {'masyarakat': 'O'},
  {'dan': 'O'},
  {'seluruh': 'O'},
  {'tindakan': 'O'},
  {'Kepolisian': 'B-ORG'},
  {'dalam': 'O'},
  {'menghadapi': 'O'},
  {'gangguan': 'O'},
  {'Kamtibmas,': 'O'},
  {'dan': 'O'},
  {'lain-lain': 'O'},
  {'-': 'O'},
  {'Jenderal': 'O'},
  {'Polisi': 'O'},
  {'Drs.': 'O'},
  {'Listyo': 'O'},
  {'Sigit': 'O'},
  {'Prabowo,': 'I-PER'},
  {'https://t.co/f8GZ6VPqPD.': 'O'},
  {'-': 'O'},
  {'Kapolri': 'O'},
  {'#PolriTVRadioPresisi': 'O'}]]